<a href="https://colab.research.google.com/github/ZarakiKanzaki/project-lunar-ML/blob/main/lunar-llm-llama2-7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training

In [1]:
#@title 🤗 AutoTrain LLM
#@markdown In order to use this colab
#@markdown - upload train.csv to a folder named `data/`
#@markdown - train.csv must contain a `text` column
#@markdown - choose a project name if you wish
#@markdown - change model if you wish, you can use most of the text-generation models from Hugging Face Hub
#@markdown - add huggingface information (token and repo_id) if you wish to push trained model to huggingface hub
#@markdown - update hyperparameters if you wish
#@markdown - click `Runtime > Run all` or run each cell individually
#@markdown - report issues / feature requests here: https://github.com/huggingface/autotrain-advanced/issues

import os
!pip install -U autotrain-advanced > install_logs.txt
!autotrain setup --colab > setup_logs.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.23.4 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers


In [1]:
import os
#@markdown ---
#@markdown #### Project Config
#@markdown Note: if you are using a restricted/private model, you need to enter your Hugging Face token in the next step.
project_name = 'lunar-llm-Llama-2-7B' # @param {type:"string"}
model_name = 'TinyPixel/Llama-2-7B-bf16-sharded' # @param {type:"string"}

#@markdown ---
#@markdown #### Push to Hub?
#@markdown Use these only if you want to push your trained model to a private repo in your Hugging Face Account
#@markdown If you dont use these, the model will be saved in Google Colab and you are required to download it manually.
#@markdown Please enter your Hugging Face write token. The trained model will be saved to your Hugging Face account.
#@markdown You can find your token here: https://huggingface.co/settings/tokens
push_to_hub = True # @param ["False", "True"] {type:"raw"}
hf_token = "hf_" #@param {type:"string"}
repo_id = "404NotF0und/lunar-llm-Llama-2-7B" #@param {type:"string"}

#@markdown ---
#@markdown #### Hyperparameters
learning_rate = 2e-4 # @param {type:"number"}
num_epochs = 1 #@param {type:"number"}
batch_size = 4 # @param {type:"slider", min:1, max:32, step:1}
block_size = 2048 # @param {type:"number"}
trainer = "sft" # @param ["default", "sft"] {type:"raw"}
warmup_ratio = 0.1 # @param {type:"number"}
weight_decay = 0.01 # @param {type:"number"}
gradient_accumulation = 4 # @param {type:"number"}
mixed_precision = "fp16" # @param ["fp16", "bf16", "none"] {type:"raw"}
peft = True # @param ["False", "True"] {type:"raw"}
quantization = "int4" # @param ["int4", "int8", "none"] {type:"raw"}
lora_r = 64 #@param {type:"number"}
lora_alpha = 16 #@param {type:"number"}
lora_dropout = 0.1 #@param {type:"number"}

os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
os.environ["REPO_ID"] = repo_id
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_EPOCHS"] = str(num_epochs)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["BLOCK_SIZE"] = str(block_size)
os.environ["WARMUP_RATIO"] = str(warmup_ratio)
os.environ["WEIGHT_DECAY"] = str(weight_decay)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["MIXED_PRECISION"] = str(mixed_precision)
os.environ["PEFT"] = str(peft)
os.environ["QUANTIZATION"] = str(quantization)
os.environ["LORA_R"] = str(lora_r)
os.environ["LORA_ALPHA"] = str(lora_alpha)
os.environ["LORA_DROPOUT"] = str(lora_dropout)

In [3]:
!git clone https://huggingface.co/datasets/404NotF0und/MtG-json-to-ForgeScribe

Cloning into 'MtG-json-to-ForgeScribe'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 2), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (16/16), 572.59 KiB | 5.25 MiB/s, done.


In [5]:
!autotrain llm \
--train \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--data-path /content/MtG-json-to-ForgeScribe/ \
--text-column text \
--lr ${LEARNING_RATE} \
--batch-size ${BATCH_SIZE} \
--epochs ${NUM_EPOCHS} \
--block-size ${BLOCK_SIZE} \
--warmup-ratio ${WARMUP_RATIO} \
--lora-r ${LORA_R} \
--lora-alpha ${LORA_ALPHA} \
--lora-dropout ${LORA_DROPOUT} \
--weight-decay ${WEIGHT_DECAY} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--quantization ${QUANTIZATION} \
--mixed-precision ${MIXED_PRECISION} \
$( [[ "$PEFT" == "True" ]] && echo "--peft" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}" )

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
> INFO    Running LLM
> INFO    Params: Namespace(version=False, text_column='text', rejected_text_column='rejected', prompt_text_column='prompt', model_ref=None, warmup_ratio=0.1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.01, max_grad_norm=1.0, add_eos_token=False, block_size=2048, peft=True, lora_r=64, lora_alpha=16, lora_dropout=0.1, logging_steps=-1, evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, mixed_precision='fp16', quantization='i

### Displaying the result

In [ ]:
# Reading the contents of the latest uploaded file
latest_file_path = '/mnt/data/logllm.log'

with open(latest_file_path, 'r') as file:
    latest_log_data = file.readlines()

# Convert the string representations of dictionaries to actual dictionaries
latest_log_dicts = [ast.literal_eval(line.strip()) for line in latest_log_data]

# Create a DataFrame from the list of dictionaries
latest_df = pd.DataFrame(latest_log_dicts)

# Filtering the DataFrame to include data from 0.06 epochs onwards
latest_filtered_df = latest_df[latest_df['epoch'] >= 0.06]

# Fit the polynomial regression model to the latest data
model.fit(latest_filtered_df[['epoch']], latest_filtered_df['loss'])

# Generating latest predictions for the trend line
latest_trend_line = model.predict(latest_filtered_df[['epoch']])

# Re-plotting with the latest filtered data and the polynomial trend line

# Setting up the figure and axis for the plot
fig, ax1 = plt.subplots()

# Plotting Loss
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='tab:red')
ax1.plot(latest_filtered_df['epoch'], latest_filtered_df['loss'], color='tab:red', label='Loss')
ax1.plot(latest_filtered_df['epoch'], latest_trend_line, color='tab:green', label='Trend Line (Loss) - Polynomial')
ax1.tick_params(axis='y', labelcolor='tab:red')
ax1.legend(loc='upper left')

# Creating a second y-axis to plot Learning Rate
ax2 = ax1.twinx()
ax2.set_ylabel('Learning Rate', color='tab:blue')
ax2.plot(latest_filtered_df['epoch'], latest_filtered_df['learning_rate'], color='tab:blue', label='Learning Rate')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.legend(loc='upper right')

# Title and layout adjustments
plt.title('Latest Loss and Learning Rate with Polynomial Trend Line from 0.06 Epochs Onwards')
plt.tight_layout()

# Displaying the updated plot
plt.show()